# Tratamento de dados de um DataFrame de Animes da Crunchyroll

## Importação das bibliotecas e carregamento do DataFrame

In [1]:
import pandas as pd
import numpy as np

# Carregamento dos dados
caminho_arquivo = 'animes.csv'
dados_animes = pd.read_csv(caminho_arquivo)

## Renomeando colunas

In [2]:
# Renomeando e preparando colunas
dados_animes.rename(columns={
    'rate': 'Avaliação do anime',
    'votes': 'Votos totais do anime',
    'episodes': 'Quantidade de episódios do anime'
}, inplace=True)

## Tratamento de dados nulos e Cálculos estatísticos

In [3]:
# Exclusão de animes sem episódios
dados_animes = dados_animes[dados_animes['Quantidade de episódios do anime'] > 0]

# Contagem total de votos
dados_animes['Contagem total de votos de todos os animes'] = dados_animes['Votos totais do anime'].sum()

# Preparação das estatísticas por gênero
colunas_genero = [col for col in dados_animes.columns if 'genre_' in col]
# Inicializar as colunas de estatísticas
dados_animes['Votos do gênero'] = 0
dados_animes['Animes do gênero'] = 0
dados_animes['Episódios do gênero'] = 0
dados_animes['Média de avaliações do gênero'] = np.nan
dados_animes['Assimetria da média do gênero'] = np.nan
dados_animes['Curtose da média do gênero'] = np.nan
dados_animes['Desvio padrão da média do gênero'] = np.nan
dados_animes['Gênero Calculado'] = ""

for genero in colunas_genero:
    genero_label = genero.replace('genre_', '').capitalize()
    grupo = dados_animes[dados_animes[genero] == 1]

    if not grupo.empty:
        dados_animes.loc[grupo.index, 'Votos do gênero'] = grupo['Votos totais do anime'].sum()
        dados_animes.loc[grupo.index, 'Animes do gênero'] = grupo.shape[0]
        dados_animes.loc[grupo.index, 'Episódios do gênero'] = grupo['Quantidade de episódios do anime'].sum()
        dados_animes.loc[grupo.index, 'Média de avaliações do gênero'] = grupo['Avaliação do anime'].mean()
        dados_animes.loc[grupo.index, 'Assimetria da média do gênero'] = grupo['Avaliação do anime'].skew()
        dados_animes.loc[grupo.index, 'Curtose da média do gênero'] = grupo['Avaliação do anime'].kurt()
        dados_animes.loc[grupo.index, 'Desvio padrão da média do gênero'] = grupo['Avaliação do anime'].std()
        dados_animes.loc[grupo.index, 'Gênero Calculado'] = genero_label

# Classificação de avaliação
dados_animes['Avaliação Categoria do anime'] = dados_animes['Avaliação do anime'].apply(lambda valor: (
    "Péssimo" if valor <= 1 else
    "Ruim" if valor <= 2 else
    "Neutro" if valor <= 3 else
    "Bom" if valor <= 4 else
    "Excelente"
))

## Reorganizando os dados

In [4]:
# Salvando o DataFrame processado para exportação
colunas_exportacao = [
    'anime', 'anime_url', 'anime_img', 'Quantidade de episódios do anime',
    'Votos totais do anime', 'Avaliação do anime', 'Avaliação Categoria do anime',
    'Contagem total de votos de todos os animes', 'Votos do gênero', 'Animes do gênero',
    'Episódios do gênero', 'Média de avaliações do gênero', 'Assimetria da média do gênero',
    'Curtose da média do gênero', 'Desvio padrão da média do gênero', 'Gênero Calculado'
]

## Exportando os dados tratados para um novo CSV

In [5]:
dados_animes_final = dados_animes[colunas_exportacao]
caminho_arquivo_processado = 'animes_para_power_bi.csv'
dados_animes_final.to_csv(caminho_arquivo_processado, index=False)

print(f"Arquivo CSV processado e pronto para uso no Power BI salvo em: {caminho_arquivo_processado}")

Arquivo CSV processado e pronto para uso no Power BI salvo em: animes_para_power_bi.csv
